In [42]:
import jieba
import pandas as pd
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [33]:
path_raw = 'hk_clean.txt'
df = pd.read_csv('hk_clean.txt',sep='\n', names=['documents'],header=None)

In [34]:
def get_corpus(fname=path_raw):
    with open(fname) as f:
        lines = iter(f)
        #lines.next()
        for line in lines:
            yield line

class CorpusFriendly(object):
    def __init__(self,fname, topK=None):
        self.topK = topK
        self.fname = fname

    def __iter__(self):
        k = 1
        corpus_memory_friendly = get_corpus(self.fname)
        for context in corpus_memory_friendly:
            #data = gensim.utils.to_unicode(data).split(',')
            seg_context = jieba.cut(context, cut_all=True)
            words = " ".join(seg_context).split()
            #print ", ".join(words)
            label = [str(k)]
            yield gensim.models.doc2vec.TaggedDocument(words, label)

            if self.topK:
                if k >= self.topK:
                    break
                k += 1

In [35]:
# corpus_train = CorpusFriendly(fname='hk_clean.txt',topK=3)
# for c in corpus_train:
#     print ", ".join(c.words)

In [36]:
df.head()

,documents
0,课题一 列强的入侵第一章中英鸦片战争乾隆晚年清朝隔世油性转衰嘉庆道光以来政治腐败经济凋敝社会...
1,从18世纪後期开始英国锐意拓展对华贸易并希望在对等的基础上和清廷交旺但清廷却自居天朝上国轻视...
2,第一次鸦片战争前中外贸易关系清朝的外交贸易态度清朝秉持传统的政治观念以天朝上国自居视海外诸国...
3,中国是一个农业社会经济自给自足因此清廷对外贸易发展并不积极并认为准许外商来华贸易只是嘉惠远人...
4,清朝对中外贸易的管理措施清初厉行海禁政策至1684年康熙帝收复台湾后才解除海禁指定广州漳州宁...


In [6]:
# for c in get_corpus('hk_clean.txt'):
#     print c

In [49]:
corpus_train = CorpusFriendly(fname='ma_clean.txt')
model = gensim.models.doc2vec.Doc2Vec(size=2000, min_count=1, iter=30, window=10, workers=6)
model.build_vocab(corpus_train)

2016-12-03 18:45:49,148 : INFO : collecting all words and their counts
2016-12-03 18:45:49,193 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2016-12-03 18:45:49,478 : INFO : collected 12907 word types and 1 unique tags from a corpus of 647 examples and 96272 words
2016-12-03 18:45:49,505 : INFO : min_count=1 retains 12907 unique words (drops 0)
2016-12-03 18:45:49,506 : INFO : min_count leaves 96272 word corpus (100% of original 96272)
2016-12-03 18:45:49,536 : INFO : deleting the raw counts dictionary of 12907 items
2016-12-03 18:45:49,536 : INFO : sample=0 downsamples 0 most-common words
2016-12-03 18:45:49,537 : INFO : downsampling leaves estimated 96272 word corpus (100.0% of prior 96272)
2016-12-03 18:45:49,537 : INFO : estimated required memory for 12907 words and 2000 dimensions: 215555100 bytes
2016-12-03 18:45:49,554 : INFO : constructing a huffman tree from 12907 words
2016-12-03 18:45:49,954 : INFO : built huffman tree with maximum node dept

In [50]:
model.train(corpus_train)

2016-12-03 18:45:51,568 : INFO : training model with 6 workers on 12907 vocabulary and 2000 features, using sg=0 hs=1 sample=0 negative=0
2016-12-03 18:45:51,568 : INFO : expecting 647 sentences, matching count from corpus used for vocabulary survey
2016-12-03 18:45:52,665 : INFO : PROGRESS: at 2.47% examples, 89395 words/s, in_qsize 2, out_qsize 0
2016-12-03 18:45:53,832 : INFO : PROGRESS: at 7.47% examples, 98119 words/s, in_qsize 9, out_qsize 0
2016-12-03 18:45:54,843 : INFO : PROGRESS: at 12.47% examples, 109762 words/s, in_qsize 9, out_qsize 0
2016-12-03 18:45:55,865 : INFO : PROGRESS: at 17.05% examples, 110918 words/s, in_qsize 12, out_qsize 0
2016-12-03 18:45:56,928 : INFO : PROGRESS: at 22.47% examples, 115814 words/s, in_qsize 9, out_qsize 0
2016-12-03 18:45:57,971 : INFO : PROGRESS: at 27.05% examples, 115454 words/s, in_qsize 11, out_qsize 0
2016-12-03 18:45:59,020 : INFO : PROGRESS: at 32.43% examples, 114403 words/s, in_qsize 12, out_qsize 0
2016-12-03 18:46:00,082 : INFO

2612490

In [51]:
model.save('gensim_doc2vec_ma_clean')

2016-12-03 18:46:57,203 : INFO : saving Doc2Vec object under gensim_doc2vec_ma_clean, separately None
2016-12-03 18:46:57,204 : INFO : not storing attribute syn0norm
2016-12-03 18:46:57,205 : INFO : not storing attribute cum_table
2016-12-03 18:46:57,206 : INFO : storing numpy array 'syn0' to gensim_doc2vec_ma_clean.syn0.npy
2016-12-03 18:46:57,377 : INFO : storing numpy array 'syn1' to gensim_doc2vec_ma_clean.syn1.npy


In [52]:
result = model.most_similar(positive=[u'毛泽东'], topn=50)
for e in result:
    print e[0], e[1]

2016-12-03 18:46:58,177 : INFO : precomputing L2-norms of word weight vectors


大会 0.487304121256
邓小平 0.426903009415
错误 0.425654470921
恩来 0.409264981747
七届 0.406742185354
中央 0.40299975872
救亡图存 0.401721507311
修正 0.397239804268
召开 0.394397318363
决定 0.389851659536
左 0.389674842358
领导 0.381504237652
4 0.371776252985
代表人 0.370344400406
报告 0.364824742079
举行 0.362404286861
过渡时期 0.357644736767
3 0.354215204716
周恩来 0.353090584278
思想 0.35283434391
严复 0.352674037218
文章 0.345772027969
两万多 0.344606101513
会议 0.343152761459
互助 0.342843621969
台下 0.342572450638
76 0.335445225239
出席 0.335164159536
李大钊 0.333106100559
他 0.332416325808
马列 0.331359565258
南昌 0.331095695496
袁世凯 0.330088376999
五个 0.326589643955
刘伯承 0.325404465199
想为 0.320692956448
汪精卫 0.31683576107
朱德 0.315772920847
三中 0.314972966909
制 0.313936114311
35 0.311945468187
二中 0.310921907425
三个代表 0.308386087418
正面 0.308221697807
根据地 0.306825041771
389 0.306708037853
如期举行 0.305099487305
主持 0.304352432489
变法 0.304013311863
10 0.303497284651


In [53]:
result = model.most_similar(positive=[u'共产'], topn=50)
for e in result:
    print e[0], e[1]

市 0.519082963467
左右 0.508009254932
资产 0.501721024513
捐款 0.490958988667
行家 0.474751919508
争先恐后 0.468380063772
积极开展 0.462161868811
党员 0.450659692287
便利 0.449199587107
果断 0.447492331266
世纪 0.443939208984
广泛开展 0.443811774254
阶级 0.438615262508
傀儡政权 0.421137571335
谈判 0.420939385891
提升 0.414148569107
适 0.411298900843
配给 0.407960355282
京畿 0.407841801643
共产党人 0.407772958279
严格 0.407211601734
联 0.405010282993
万股 0.403754293919
用活 0.403723239899
配上 0.397625029087
毁坏 0.396900743246
马克思主义 0.395297348499
聚会 0.393555939198
七次 0.39193046093
升华 0.39175003767
相比 0.39160245657
收 0.390299022198
逆差 0.388421416283
国际金融 0.387688845396
贸易逆差 0.385262787342
极为 0.384791254997
七 0.384758532047
第一次 0.381829559803
大力开展 0.380911022425
割让 0.380042076111
急切 0.379683464766
年限 0.377444684505
下旬 0.376877367496
尖锐 0.375366687775
七条 0.374831289053
合作 0.37399405241
无产 0.373658210039
法规 0.369240820408
邮政局 0.368238627911
一三 0.367492556572
